# Setup

### **WSL/Mac (Intel) users**:
`mamba install -c bioconda hisat2 samtools subread salmon`

### **Mac (M1+) users**:
The following cell downloads and installs the necessary software. It may take a few minutes to complete.

http://daehwankimlab.github.io/hisat2/download/#version-hisat2-221


In [ ]:
%%bash
curl -O https://genome-idx.s3.amazonaws.com/hisat/mm10_genome.tar.gz
tar -xzf mm10_genome.tar.gz
rm mm10_genome.tar.gz

if [ $(arch) != "arm64" ]; then
    curl -LSsO https://cloud.biohpc.swmed.edu/index.php/s/zMgEtnF6LjnjFrr/download
    tar xzf download
    
    curl -LSsO https://github.com/ShiLab-Bioinformatics/subread/releases/download/2.0.2/subread-2.0.2-macOS-x86_64.tar.gz
    tar xzf subread-2.0.2-macOS-x86_64.tar.gz

    if ! grep -q 'eval "$(/opt/homebrew/bin/brew shellenv)"' $HOME/.zprofile; then
        echo 'eval "$(/opt/homebrew/bin/brew shellenv)"' >> $HOME/.zprofile
    fi
    eval "$(/opt/homebrew/bin/brew shellenv)"
    brew install samtools
fi

Ok - so we have an idea of how to make data experimentally from the last lecture, but now we have to think about how to analyze some data!

I'm going to steal from the _lovely_ [Galaxy project](https://galaxyproject.org/) for this tutorial - specifically here: [Transcriptomics](https://training.galaxyproject.org/training-material/topics/transcriptomics/)

We are going to work from RNA-seq data from [Fu et al.](https://www.nature.com/articles/ncb3117) where they sequenced basal and luminal cells from breast tissue of nulliparous, pregnant, or lactating female mice.

The first thing we are going to do is to grab a "FASTQ" file.

FASTQ [Wiki](https://en.wikipedia.org/wiki/FASTQ_format) is a plain text file that encodes for a DNA sequence - along with how confident (quality score) we are that the read is correct.

For the purposes of instruction here and to make sure it runs smoothly, we are going to use a downsampled dataset that is only 1000 reads from each sample, hosted at [Zenodo](https://zenodo.org/record/4249555)

First, let's setup a table of the data


In [ ]:
import pandas as pd

In [ ]:
meta = pd.DataFrame(
    data={
        "sampleid": [
            "MCL1-DL",
            "MCL1-DK",
            "MCL1-DJ",
            "MCL1-DI",
            "MCL1-DH",
            "MCL1-DG",
            "MCL1-LF",
            "MCL1-LE",
            "MCL1-LD",
            "MCL1-LC",
            "MCL1-LB",
            "MCL1-LA",
        ],
        "celltype": [
            "basal",
            "basal",
            "basal",
            "basal",
            "basal",
            "basal",
            "luminal",
            "luminal",
            "luminal",
            "luminal",
            "luminal",
            "luminal",
        ],
        "mousetype": [
            "lactate",
            "lactate",
            "virgin",
            "virgin",
            "pregnant",
            "pregnant",
            "lactate",
            "lactate",
            "pregnant",
            "pregnant",
            "virgin",
            "virgin",
        ],
        "url": [
            "https://zenodo.org/record/4249555/files/SRR1552455.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552454.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552453.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552452.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552451.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552450.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552449.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552448.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552447.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552446.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552445.fastq.gz",
            "https://zenodo.org/record/4249555/files/SRR1552444.fastq.gz",
        ],
    }
)

In [ ]:
meta["destfile"] = meta.sampleid + ".fq.gz"
meta

Ok - got that in. Now let's download the data, making a directory called "timplab_rnaseq_data" in your current directory


In [ ]:
%%bash
mkdir -p timp_rnaseq_data

In [ ]:
from urllib.request import urlretrieve

for url, dst in zip(meta["url"], meta["destfile"]):
    # Let's download the data
    urlretrieve(url, "timp_rnaseq_data/" + dst)
    print("Downloaded", dst)

### Examine FASTQ

Ok - so we have downloaded a bunch of FASTQ files - what do they look like


In [ ]:
import gzip

with gzip.open("timp_rnaseq_data/MCL1-DK.fq.gz", "rt") as f:
    for _ in range(8):
        print(f.readline().strip())

In [ ]:
%%bash
# How to do this in bash
gunzip -cq timp_rnaseq_data/MCL1-DK.fq.gz | head -n 8

Using `gunzip` because it's a compressed text file - we can look at just the first four lines of the FASTQ

- The first line is an `@` followed by a read name and optional description.\
- The second line is the actual DNA sequence read (i.e. "A", "C", "G", "T")
- The third line is generally just a `+`, though in this case they have repeated the read name and description
- The fourth line is an ASCII encoding of the "PHRED" quality score - a log_10 score fo the quality score. In this case the front of the read is a lower quality (B=33, \@=31) and most of the read higher score (I=40, J=41). A quick [PHRED Lookup Table](https://support.illumina.com/help/BaseSpace_OLH_009008/Content/Source/Informatics/BS/QualityScoreEncoding_swBS.htm) is here from Illumina.

### Plot FASTQ qscore plot?

Let's look at what the overall quality scores are for this FASTQ, using the package "pyfastx"


In [ ]:
%%bash
pip install pyfastx

In [ ]:
# Load pyfastx package and fastq
import pyfastx

myfq = pyfastx.Fastq("timp_rnaseq_data/MCL1-DK.fq.gz")

Ok - so we have a FASTQ file here with 1000 reads (because we cut it to make it managable) and that's 100 "cycles" of sequencing long, i.e. we have 100 A,C,G,or T in each read.


Here's a quick look at the DNA sequences and quality scores, like we did from gunzip in the bash chunk above.


In [ ]:
print(myfq)

for i in range(5):
    print(i)
    print(myfq[i].name)
    print(myfq[i].seq)
    print(myfq[i].qual)

Let's plot the quality scores for just the first 5 reads.


In [ ]:
# Using matplotlib
import matplotlib.pyplot as plt

for i in range(5):
    plt.plot(myfq[i].quali, label=myfq[i].name)
plt.legend()
plt.show()

### Using plotnine

In [ ]:
import plotnine as pn

# Create a DataFrame from the quality scores
# Reset the index to a new column
quals = pd.DataFrame({myfq[i].name: myfq[i].quali for i in range(5)}).reset_index()
quals

In [ ]:
# Reshape the DataFrame to long format
quals = quals.melt(id_vars="index", var_name="Sample", value_name="Quality")
print(quals)
# Create the plot using plotnine
pn.ggplot(quals, pn.aes(x="index", y="Quality", color="Sample")) + pn.geom_line()

Note a couple of things - the beginning quality score is sometimes terrible, and the overall score drops through the read towards the end. This is frequently solved by "trimming" the reads to get the subset of high quality results we trust.

But what was the overall quality score of the reads?


In [ ]:
import statistics

perread = pd.DataFrame({"qual": [statistics.mean(read.quali) for read in myfq]})
print(perread.iloc[:10])

# Let's plot these as a "density" plot - sort of a smoothed histogram
pn.ggplot(perread, pn.aes(x="qual")) + pn.geom_density()

So _overall_ our quality scores are real good!

### Align with Hisat2

Ok - so we have the FASTQ. Now we want to "align" it against a reference genome. We are going to do this with [HiSat2](http://daehwankimlab.github.io/hisat2/) which is an aligner that is designed to work with sequencing data, specifically RNA sequencing data in this case. We'll also pull in samtools to process other things further

Installing it . . . . may be a bit of a trick for you, I suggest using conda but you will have trouble if you try to do it on an M1 mac unless using Rosetta to fake a Intel chip.


Then we need to pull down a "reference index" which uses a genome reference (in this case mouse(mm10)) to align against. The index is a specific file to the sequence aligner that uses the "raw" genome sequence to generate rapid alignments, like the index of a book allows you to look up information more rapidly.


In [ ]:
%%bash
mkdir -p timp_rnaseq_align

Then we are going to align all our data, using a for loop to loop through all the FASTQ files we downloaded. I'm doing this with a system call to the hisat2 command, as well as to samtools:


In [ ]:
i = 0
print(meta.sampleid[i])
print("Here is the bash command")
print(f"hisat2 -p 4 -x mm10/genome -U timp_rnaseq_data/{meta.destfile[i]} -S timp_rnaseq_align/{meta.sampleid[i]}.sam")

In [ ]:
%%bash
if [ $(arch) != "arm64" ]; then
    hisat2 -p 4 -x mm10/genome -U timp_rnaseq_data/MCL1-DL.fq.gz -S timp_rnaseq_align/MCL1-DL.sam
fi

In [ ]:
%%bash
# For Mac users
if [ $(arch) == "arm64" ]; then
    hisat2-2.2.1/hisat2 -p 4 -x mm10/genome -U timp_rnaseq_data/MCL1-DL.fq.gz -S timp_rnaseq_align/MCL1-DL.sam
fi

As you can see from the output, we get a report on how the reads aligned - it tells us that the reads were unpaired (which we knew) and a small percent of the reads don't align at all, the vast majority align once, and then a couple align multiple times.

- Reads that don't align we can toss, could be contamination, could just be gaps in assembly. As long as these are low it's not a huge concern.
- Reads that align multiple times could be in repetitive sequences or genes (since RNA-seq) with high homology. These are more of a concern to lose, but it's a largely unavoidable problem with short reads.
- The "uniquely" aligned reads are our meat and potatoes.

### SAM files

Let's take a quick look at the alignment file


In [ ]:
%%bash
tail -n5 timp_rnaseq_align/MCL1-DL.sam

I took the last 5 lines in this case because the beginning has a header that we aren't going to get into (with `tail`)

The SAM/BAM file spec is avilable [here](https://samtools.github.io/hts-specs/SAMv1.pdf).

Essentially it's a tab-separated text file with columns as:

- Sequence read name
- Flags (i.e. multiple alignment, etc)
- Reference sequence it's aligned to (in this case the chromomsome)
- Map Quality score (How confident it is of correct alignment on a log scale, 0 is bad, high numbers good)
- CIGAR string - this specifies differences from the reference, with M being matches (and often mismatches), D deletions and I insertions
- RNEXT is used to specify the reference sequence (chromosome) for a paired read if there is one
- PNEXT is used to specify the position of the paired read
- TLEN is the length of the molecule - again if you have paired reads you want to know the distance between the paired start and end
- SEQ is the sequence of the read (from the FASTQ)
- QUAL is the quality of the read (from the FASTQ)
- Extra Alignment Tags

Let's look at some stats of this SAM file


In [ ]:
%%bash
samtools flagstat timp_rnaseq_align/MCL1-DL.sam

Looking at this, it's the same info, but perhaps a bit easier to parse and putting a nice "Ranges" ojbect for the alignments.

Note we have more than 1000 alignments - that's because we had those reads with multiple alignments! But they'll have the same "qname"

### Gene alignment

Ok - finally let's look at how many of our alignments go to _genes_

subread has a "in-built" annotation - mine is in `~/miniforge3/envs/quantneuro/annotation`


In [ ]:
%%bash
if [ $(arch) != "arm64"]; then
    featureCounts -a $CONDA_PREFIX/annotation/mm10_RefSeq_exon.txt -F SAF -o timp_rnaseq_align/MCL1-DL.counts timp_rnaseq_align/MCL1-DL.sam
fi

In [ ]:
%%bash
# For Mac users
if [ $(arch) == "arm64" ]; then
    subread-2.0.2-macOS-x86_64/bin/featureCounts -a subread-2.0.2-macOS-x86_64/annotation/mm10_RefSeq_exon.txt -F SAF -o timp_rnaseq_align/MCL1-DL.counts timp_rnaseq_align/MCL1-DL.sam
fi

So most of our reads are actually assigned to a gene - this is great.

We can even see what gene and how many counts - then filter out genes with no counts, and sort by number of counts


In [ ]:
genecounts = pd.read_csv(
    "timp_rnaseq_align/MCL1-DL.counts", delimiter="\t", comment="#"
)

# Filter out only genes with >0 counts
genecounts = genecounts[genecounts["timp_rnaseq_align/MCL1-DL.sam"] > 0]

# Sort on highest counts
genecounts.sort_values(by="timp_rnaseq_align/MCL1-DL.sam", ascending=False)

And look! GeneID 11475 - which is [https://www.ncbi.nlm.nih.gov/nuccore/NM_007392.3] actin! A very highly expressed gene, so this makes perfect sense.
